# 🧾 Product Sales Analysis — Pens & Printers

![](0_projectimage.png)



Pens and Printers, founded in 1984, provides high-quality office products to large organizations. They are a trusted supplier of items ranging from pens and notebooks to desk chairs and monitors, selling products made by other companies rather than producing their own.

Recently, Pens and Printers launched a new office stationery line and are adapting their sales strategies to ensure effective promotion. Launching a new product line is costly, so they have tested three sales approaches—email, phone, and a combination of both—to quickly determine the most effective method.




In [87]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset 
sales = pd.read_csv('https://s3.amazonaws.com/talent-assets.datacamp.com/product_sales.csv')
sales 


,week,sales_method,customer_id,nb_sold,revenue,years_as_customer,nb_site_visits,state
0,2,Email,2e72d641-95ac-497b-bbf8-4861764a7097,10,NaN,0,24,Arizona
1,6,Email + Call,3998a98d-70f5-44f7-942e-789bb8ad2fe7,15,225.47,1,28,Kansas
2,5,Call,d1de9884-8059-4065-b10f-86eef57e4a44,11,52.55,6,26,Wisconsin
3,4,Email,78aa75a4-ffeb-4817-b1d0-2f030783c5d7,11,NaN,3,25,Indiana
4,3,Email,10e6d446-10a5-42e5-8210-1b5438f70922,9,90.49,0,28,Illinois
...,...,...,...,...,...,...,...,...
14995,4,Call,17267b41-d048-4346-8b90-7f787690a836,10,50.82,0,22,Pennsylvania
14996,5,Call,09e10d6f-4508-4b27-895e-4db11ce8302b,10,52.33,1,27,Kansas
14997,1,Call,839653cb-68c9-48cb-a097-0a5a3b2b298b,7,34.87,4,22,West Virginia
14998,6,Call,e4dad70a-b23b-407c-8bd3-e32ea00fae17,13,64.90,2,27,New Jersey


In [88]:
# Data types 
print(sales.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   week               15000 non-null  int64  
 1   sales_method       15000 non-null  object 
 2   customer_id        15000 non-null  object 
 3   nb_sold            15000 non-null  int64  
 4   revenue            13926 non-null  float64
 5   years_as_customer  15000 non-null  int64  
 6   nb_site_visits     15000 non-null  int64  
 7   state              15000 non-null  object 
dtypes: float64(1), int64(4), object(3)
memory usage: 937.6+ KB
None


In [89]:
print(sales.groupby('week').size()) # 1. week
print(sales['sales_method'].unique()) # 2. sales_method
print(sales['customer_id'].nunique()) # 3. customer_id
print(sales['nb_sold'].agg(['min', 'max'])) # 4. nb_sold
print(sales['revenue'].agg(['min', 'max'])) # 5. revenue
print(sales['years_as_customer'].agg(['min', 'max'])) # 6. years_as_customer
print(sales['nb_site_visits'].agg(['min', 'max'])) # 7. nb_site_visits
print(sales['state'].unique()) # 8. state
print(sales['state'].nunique())



week
1    3721
2    2491
3    2411
4    2575
5    2574
6    1228
dtype: int64
['Email' 'Email + Call' 'Call' 'em + call' 'email']
15000
min     7
max    16
Name: nb_sold, dtype: int64
min     32.54
max    238.32
Name: revenue, dtype: float64
min     0
max    63
Name: years_as_customer, dtype: int64
min    12
max    41
Name: nb_site_visits, dtype: int64
['Arizona' 'Kansas' 'Wisconsin' 'Indiana' 'Illinois' 'Mississippi'
 'Georgia' 'Oklahoma' 'Massachusetts' 'Missouri' 'Texas' 'New York'
 'Maryland' 'California' 'Tennessee' 'Pennsylvania' 'North Dakota'
 'Florida' 'Michigan' 'North Carolina' 'Hawaii' 'Colorado' 'Louisiana'
 'Virginia' 'New Mexico' 'Arkansas' 'Alaska' 'Oregon' 'New Hampshire'
 'Ohio' 'New Jersey' 'Connecticut' 'Iowa' 'Montana' 'Washington'
 'Kentucky' 'Alabama' 'Nebraska' 'South Carolina' 'Minnesota'
 'South Dakota' 'Delaware' 'Maine' 'Utah' 'West Virginia' 'Vermont'
 'Rhode Island' 'Nevada' 'Idaho' 'Wyoming']
50


In [90]:
# 2. sales_method - standardize capitalization and strip spaces
sales['sales_method'] = sales['sales_method'].str.strip().str.title()

# Automatic mapping based on substring
def standardize_method(x):
    if 'Email' in x and 'Call' in x:
        return 'Email + Call'
    elif 'Email' in x:
        return 'Email'
    elif 'Call' in x:
        return 'Call'
    else:
        return x  # Keep any unknown values for auditing

sales['sales_method'] = sales['sales_method'].apply(standardize_method)

# Check categories on sales_method after cleaning
print("Categories after cleaning:", sales['sales_method'].unique())

# 5. revenue - Check the proportion of missing values
missing_ratio = sales['revenue'].isna().sum() / len(sales)
print(f"Missing values in 'revenue': {missing_ratio:.2%}")

# Impute missing values using the median of each group (nb_sold, sales_method)
sales.loc[:, 'revenue'] = sales.groupby(['nb_sold', 'sales_method'])['revenue']\
                              .transform(lambda x: x.fillna(x.median()))

# Confirm there are no missing values left
print("Missing values after imputation:", sales['revenue'].isna().sum())


# 6. years_as_customer -keep only rows within the valid range (0-41)
sales = sales[(sales['years_as_customer'] >= 0) & 
              (sales['years_as_customer'] <= 41)]

# Check range on years_as_customer after cleaning
print("Min:", sales['years_as_customer'].min(), "Max:", sales['years_as_customer'].max())

# Check the final shape after data cleaning and validation
sales.describe()




Categories after cleaning: ['Email' 'Email + Call' 'Call']
Missing values in 'revenue': 7.16%
Missing values after imputation: 0
Min: 0 Max: 39


,week,nb_sold,revenue,years_as_customer,nb_site_visits
count,14998.00000,14998.000000,14998.000000,14998.000000,14998.000000
mean,3.09828,10.084678,95.714353,4.959261,24.990799
std,1.65649,1.812334,48.384616,5.011237,3.501100
min,1.00000,7.000000,32.540000,0.000000,12.000000
25%,2.00000,9.000000,52.760000,1.000000,23.000000
50%,3.00000,10.000000,90.000000,3.000000,25.000000
75%,5.00000,11.000000,108.350000,7.000000,27.000000
max,6.00000,16.000000,238.320000,39.000000,41.000000


# **Data Validation**

The dataset contains **15000 rows and 8 columns** before cleaning and validation.

- week: Values range from 1 to 6, consistent with the data description, with no missing or negative values. No cleaning is required.
  
- sales_method: Contains 5 distinct labels instead of the expected 3, due to inconsistent capitalization and spacing. After cleaning, the three standardized categories used were: Email, Call, and Email + Call.

- customer_id: All entries are non-missing and unique, as expected. No cleaning is necessary.

- nb_sold: Non-negative integers, no missing values, consistent with the data description.

- revenue: Contains approximately **7.16% missing values**; all non-missing values are non-negative. Missing values were imputed using a group-wise median within each (nb_sold, sales_method) group to preserve distribution. Because the revenue data is right-skewed, the median is a more robust measure of central tendency than the mean.

- years_as_customer: All values are non-negative integers, and should fall between 0 and 41 (from 1984 to present). There are two entries (47, 63 years) that are not consistent with expectations and have been excluded. 

- nb_site_visits: All values are non-negative integers without missing data, as expected. No cleaning is required.

- state: No missing values; all values are present and correspond to 50 unique U.S. states, suggesting good data quality. No cleaning needed.

Following data validation and imputation, the final dataset consists of **14998 rows and 8 columns**, with no remaining missing values.

# **Exploratory Data Analysis (EDA)**

## Customer Distribution by Sales Method

Over the last six weeks, Email was used most widely (**49.8%**), while call-based methods were more limited due to resource intensity. The Call method accounted for about one-third (**33.1%**) of customers, while the combined **Email + Call** method reached the smallest segment of the customers (**17.1%**).


![1_barplot_customer_by_sales_methods](1_barplot_customer_by_sales_methods.png)


## Revenue Distribution (Overall)

Both the histogram and boxplot show that overall revenue across all methods is right-skewed, reflecting distinct customer spending patterns. The median revenue is **90 USD**, lower than the mean of  **96 USD**, due to a small number of high-spending customers (top 10% exceed **183 USD**). Most customers generate between **53 USD** and **108 USD**.

![2_his_boxplot_revenue](2_his_boxplot_revenue.png)


## Revenue by Sales Method

The boxplots illustrate distinct revenue patterns, indicating that customer responses vary significantly depending on the sales method used. 

- The Email + Call approach clearly outperforms the others, delivering the highest typical revenue per customer at **184 USD**, with values reaching up to **238 USD**. While it shows greater variability, this method demonstrates strong potential and is well-suited for **selective targeting of high-value customers**.

- The Email method generated a median revenue of **96 USD** per customer, with **consistently reliable results**—most customers spent between **88 USD** (25th percentile) and **105 USD** (75th percentile).

- In contrast, the Call method delivered the lowest returns, with a median revenue of just **49 USD** per customer, highlighting its limited effectiveness.

These patterns suggest that while the **Email + Call** method drives the highest revenue from high-value customers, its limited use (17% of customers) points to **targeted potential rather than broad scalability**. The **Email-only** method provides **strong and consistent performance** with minimal effort, making it ideal for wide-scale outreach. In contrast, the **Call-only** approach delivers the **lowest returns** despite requiring the most team resources, indicating it is the **least effective strategy** overall.


![3_boxplot_revenue_by_sales_method](3_boxplot_revenue_by_sales_method.png)

## Revenue Over Time by Method

Over the six-week campaign, the line plot exhibited **positive growth in median revenue across all sales methods**:
The **Email + Call** increase by **77%** (from **129 USD** in week 1 to **227 USD** in week 6). The **Email-only** grew **50%** (from **87 USD** to **130 USD**). The **Call-only** method increased **+87%** (**from 35 USD** to **67 USD** ), despite remaining the lowest in absolute terms. These trends indicate that multi-touch engagement sustains momentum most effectively.

![4_lineplot_weekly_revenue_by_sales_method](4_lineplot_weekly_revenue_by_sales_method.png)

# Customer Insights

## Customer loyalty

On average, customers have been with the company for 5 years, but the median tenure is 3 years, indicating a skew toward newer clients. Most customers have a tenure of 1 year, while a small number of long-term customers pull the average upward. Customer loyalty showed a **negligible negative correlation (r = -0.07)**.


![5_hist_years_as_customer](5_hist_years_as_customer.png) 

## Customer distribution across USA states:

Customer distribution is concentrated in a few key states—California (12.5%), Texas (7.9%), New York (6.4%), and Florida (6.0%)—which together make up over 32% of customers. This distribution closely mirrors the overall population spread across the United States, highlighting natural regional market priorities. 

Median revenue does not significantly vary by state, indicating consistent customer value nationwide.

![5_hist_state](5_hist_state.png)

## Customer Engagement Impact

**Customer engagement**, measured by the number of site visits in the last 6 months, showed a **modest positive correlation** with revenue (**r = 0.33**), suggesting that more engaged customers may spend slightly more.

However, the **sales method**  is by far the strongest determinant of revenue differences. The variation in revenue between methods is far more significant than differences caused by loyalty, state or engagement.

While engagement alone is not a strong predictor, it may still provide **strategic value** for **future targeting and personalization efforts**.


# Business Metrics 

To effectively track and optimize sales performance across methods, I recommend monitoring three business metrics weekly (based on median values):

1. **Revenue per Customer**: will highlight which approach generates the greatest revenue impact;
    
    
2. **Units Sold per Customer**: will uncover customer purchasing depth and identify upselling opportunities;
    
       
3. **Revenue per Unit Sold**: will assess the value of each transaction, revealing whether methods drive higher-value purchases through premium products;

Together, these metrics balance efficiency and effectiveness, providing a clear, data-driven basis for adjusting sales tactics and allocating resources.

Here's a snapshot of current performance based on median values for each method:

| **Sales Method** | **Revenue per Customer** | **Units Sold per Customer** | **Revenue per Unit** |
|------------------|:------------------------:|:----------------------------:|:---------------------:|
| Call             | $49                      | 10                           | $5                    |
| Email            | $96                      | 10                           | $10                   |
| Email + Call     | $184                     | 12                           | $15                   |



Customers reached through the **Email + Call** approach purchased more units (**12 units**, median) and generated the **highest revenue per unit** (15 USD), indicating they may be buying either **higher-priced products** or **larger quantities**.

The **Email-only** method shows solid performance: it matches the Call method in units sold (10) but nearly **doubles the revenue per unit**, making it a **highly cost-effective and scalable strategy**.

In contrast, the **Call-only** method delivers the lowest revenue per customer and per unit, despite similar units sold. This suggests **lower transaction value** or **reduced effectiveness** in influencing purchases, especially when considering its higher time and resource cost.


# Recommendations

Based on the analysis, the following strategic recommendations aim to **maximize sales performance** and **optimize resource allocation**:


- **Discontinue Call-Only Method**:
Requires 30 minutes per customer, yet delivers the lowest revenue. Reallocating those team resources will bolster higher-return methods. 


- **Prioritize Email + Call for High-Potential Customers**:
As it delivers the **highest revenue** per customer and per unit with **moderate effort** (10 minutes per customer).


- **Use Email for Broad Outreach**:
Leverage the Email-only method for broad outreach, as it is a **scalable strategy** providing **solid returns with minimal effort**."


- **Invest in Engagement**:
Revenue increases modestly with customer site visits. Drive engagement through targeted campaigns, newsletters, or promotions to support sales.


- **Monitor Business Metrics weekly**:
Track the three key metrics weekly—**revenue per customer, units sold per customer, and revenue per unit**—to evaluate ongoing performance and guide timely adjustments.


- **Improve Data Quality and Collection**:
by implementing mandatory revenue entry validation to reduce missing values below 1%, ensuring cleaner data for forecasting and analysis.

Aligning sales methods with customer value and resource availability will enable the company to **balance revenue growth with operational efficiency effectively**.

